# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your GT login and the GT logins of any of your collaborators below. (The GT logins are worth 1 point per notebook, so don't miss the opportunity to get a free point!)

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# Introduction to cellular automata

The following exercises accompany the class slides on Wolfram's 1-D nearest neighbor cellular automata model. You can download a copy of those slides here: [PDF (6 MiB)](https://t-square.gatech.edu/access/content/group/gtc-239f-fc11-5690-9dae-2dc96b59f372/cx4230-sp17--08--lab3.pdf)

## Setup

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse

In [ ]:
import matplotlib.pyplot as plt # Core plotting support
%matplotlib inline

def show_grid (grid, **args):
    plt.matshow (grid, **args)
    
# Demo of `show_grid`
grid = np.array ([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])
show_grid (grid)
plt.colorbar ()

## Wolfram's 1-D near-neighbor CA

Let's evolve a 1-D region of length `N` over `T` time steps.

Start by creating a 2-D Numpy array (or _matrix_) `X[0:N, 0:T]`, which will eventually hold the sequence of all state changes over time. Our convention will be to store either a `0` or a `1` value in every cell.

In [ ]:
N = 11
T = 20

X = np.zeros ((N, T), dtype=int)  # X[i, t] == cell i at time t

show_grid (X.T, cmap='hot') # Transpose!

**Exercise 1** (2 points). As the initial state of the 1-D system, let's put a single `1` bit at or close to the center at time 0.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

show_grid (X, cmap='hot')

## Sparse matrices

Suppose you are given a 1-D neighborhood as a 3-bit pattern, `011`$_2$. This value is the binary representation of the decimal value, $(2^2 \cdot 0) + (2^1 \cdot 1) + (2^0 \cdot 1) = 3$. More generally, given a 3-bit string, $b_2b_1b_0$, let its _neighborhood index_ be the decimal integer $k$ such that

$$
  k \equiv (4 \cdot b_2) + (2 \cdot b_1) + (1 \cdot b_0).
$$

Given one of Wolfram's rules, you could then build a lookup table to convert every possible neighborhood index into the corresponding `0` or `1` state.

To implement this idea, try this notional trick from linear algebra. Let $\vec{x}$ denote the 1-D grid of $n$ cells, represented as a _vector_ of $n$ bits,

$$\begin{eqnarray}
  \vec{x}
    & = &
      \left(\begin{array}{c}
        x_0 \\
        x_1 \\
        \vdots \\
        x_{n-1}
      \end{array}\right),
\end{eqnarray}$$

where $x_i \in \{0, 1\}$.

From this vector, you can enumerate all neighborhood indices using a _(sparse) matrix-vector product_. Let $k_i$ denote the neighborhood index of cell (bit) $x_i$. Then,

$$\begin{eqnarray}
  k_0 & = & 2 x_0 + x_1 \\
  k_1 & = & 4 x_0 + 2 x_1 + x_2 \\
  k_2 & = & 4 x_1 + 2 x_2 + x_3 \\
      & \vdots & \\
  k_i & = & 4 x_{i-1} + 2 x_i + x_{i+1} \\
      & \vdots & \\
  k_{n-2} & = & 4 x_{n-3} + 2 x_{n-2} + x_{n-1} \\
  k_{n-1} & = & 4 x_{n-2} + 2 x_{n-1}
\end{eqnarray}$$

This system of equations can be written in matrix form as $\vec{k} \equiv A \cdot \vec{x}$, where

$$
\vec{k} \equiv \left(\begin{array}{c}
                 k_0 \\
                 k_1 \\
                 k_2 \\
                 \vdots \\
                 k_i \\
                 \vdots \\
                 k_{n-2} \\
                 k_{n-1}
               \end{array}\right)
=
  \underbrace{\left(\begin{array}{cccccccc}
    2 & 1 &   &        &   &        &   & \\
    4 & 2 & 1 &        &   &        &   & \\
      & 4 & 2 & 1      &   &        &   & \\
      &   &   & \ddots &   &        &   & \\
      &   &   &    4   & 2 & 1      &   & \\
      &   &   &        &   & \ddots &   & \\
      &   &   &        &   &    4   & 2 & 1 \\
      &   &   &        &   &        & 4 & 2
  \end{array}\right)}_{\equiv A}
\cdot
  \underbrace{\left(\begin{array}{c}
                 x_0 \\
                 x_1 \\
                 x_2 \\
                 \vdots \\
                 x_i \\
                 \vdots \\
                 x_{n-2} \\
                 x_{n-1}
               \end{array}\right)}_{= \vec{x}}.
$$

The matrix $A$ is _sparse_ because it is mostly zero.

> Sparsity does not have a precise formal definition. However, one typically expects that the number of non-zeros in $n \times n$ sparse matrix $A$ is $\mathrm{nnz}(A) = \mathcal{O}(n)$.

In fact, $A$ has a more specific structure: it is _tridiagonal_, meaning that all of its non-zero entries are contained in the diagonal of $A$ plus the first sub- and super-diagonals.

Scipy has an especially handy function, `scipy.sparse.diags()`, which can easily construct sparse matrices consisting only of diagonals:  http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.diags.html#scipy.sparse.diags

**Exercise 2** (2 points). Read the documentation for this routine and construct $A$ as defined above. Store your result in a variable named, `A`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert type (A) is sp.sparse.dia.dia_matrix

A_dense = A.toarray ()
print ("=== A (dense) ===", A_dense, sep="\n")

# Test values
A_dense_diag_sub = np.diag (A_dense, -1)
A_dense_diag = np.diag (A_dense)
A_dense_diag_sup = np.diag (A_dense, 1)

assert (A_dense_diag_sub == 4).all ()
assert (A_dense_diag == 2).all ()
assert (A_dense_diag_sup == 1).all ()
assert ((A_dense
         - np.diag (A_dense_diag)
         - np.diag (A_dense_diag_sub, -1)
         - np.diag (A_dense_diag_sup, 1)) == 0).all ()

print ("\n(Passed.)")

As a sanity check, let's multiply $A$ by the initial 1-D grid. Denote this initial grid mathematically as $\vec{x}(t=0)$, which is just the first column of the array `X`, i.e., `X[:, 0]`.

**Exercise 3** (2 point). Compute $A \cdot \vec{x}(0)$ by hand.

YOUR ANSWER HERE

Let's check your answer using the Python code below to compute $\vec{k}(0)$. It uses the `A` object's `dot()` member function.

In [ ]:
K_0 = A.dot (X[:, 0])
print (X[:, 0])
print (K_0)

**Exercise 4** (3 points). Recall that the rule number, for neighborhoods of size 3, is an integer between $0$ and $2^{2^3} -1 = 255$, inclusive. Its bit pattern determines which neighborhood patterns map to which states.

Complete the following function: given a rule number, it should build and return a lookup table, `bits[:]`, that maps a neighborhood index `k` in `[0, 8)` to the output bit `bits[k]`. For example:

```python
    gen_rule_bits (90)[::-1] == np.array ([0, 1, 0, 1, 1, 0, 1, 0])
    gen_rule_bits (150)[::-1] == np.array ([1, 0, 0, 1, 0, 1, 1, 0])
```

Note the `[::-1]` in this example, which aesthetically places the most significant bit first.

Your function should have this signature:

```python
def gen_rule_bits (rule_num):
    """
    Computes a bit lookup table for one of Wolfram's 1-D
    cellular automata (CA), given a rule number.
    """
    assert (0 <= rule_num < 256)
    ... # Build bits[:8]
    return bits
```

> Hint: Check out this handy [NumPy function](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.binary_repr.html).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert (gen_rule_bits (90)[::-1] == np.array ([0, 1, 0, 1, 1, 0, 1, 0])).all ()
assert (gen_rule_bits (150)[::-1] == np.array ([1, 0, 0, 1, 0, 1, 1, 0])).all ()

**Exercise 5** (1 point). Write some code to compute the complete state at time 1, `X[:, 1]`.

In [ ]:
RULE = 90
RULE_BITS = gen_rule_bits (RULE)

# YOUR CODE HERE
raise NotImplementedError()

print ("Rule:", RULE, "==>", RULE_BITS[::-1])
print ("x(0):", X[:, 0])
print ("k(0):", K0)
print ("==>\nx(1):", X[:, 1])

In [ ]:
assert (X[:, 1] == np.array ([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0])).all ()

**Exercise 6** (4 points). Complete the following function, which runs a 1-D `n`-cell CA for `t_max` time steps, given an initial state `x0` and a rule number `rule_num`.

In [ ]:
def run_ca (rule_num, n, t_max, x0=None):
    bits = gen_rule_bits (rule_num)
    cells = np.zeros ((n, t_max), dtype=int)
    
    # Initial condition (default: centered impulse)
    if not x0:
        cells[int (n/2), 0] = 1
    else:
        cells[:, 0] = x0
        
    cells2idx = sp.sparse.diags ([4, 2, 1], [-1, 0, 1], \
                                 shape=(n, n), dtype=int)
    
    for t in range (1, t_max):
        # YOUR CODE HERE
        raise NotImplementedError()
    return cells

Check your results against these patterns: https://commons.wikimedia.org/wiki/Elementary_cellular_automata

In [ ]:
# Some test code:
def irun_ca (rule_num=90, n=100, t_max=100):
    show_grid (run_ca (rule_num, n, t_max).transpose (), cmap='hot')
    
irun_ca (90) # Try 90, 169, and 37

Incidentally, we can create a little widget to create an interactive visualization of these 1-D automata.

In [ ]:
from ipywidgets import interact

interact (irun_ca
          , rule_num=(0, 255, 1)
          , n=(10, 100, 10)
          , t_max=(10, 100, 10))